In [3]:
import numpy as np
from matplotlib import pyplot as plt

from dftpy.ions import Ions
from dftpy.functional import Functional
from dftpy.formats import io

from ase.io import read
from ase.lattice.spacegroup import Spacegroup
from ase.lattice.spacegroup import crystal
from ase.build import bulk
from scipy.optimize import minimize

from dftpy.ions import Ions
from dftpy.functional import Functional
from dftpy.formats import io
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional, ExternalPotential
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.optimization import Optimization

/opt/homebrew/lib/python3.11/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')


In [4]:
path_pp='/Users/valeria/Documents/PP/ofpp/EAC/upf/blps/'
file='si.lda.upf'
PP_list = {'Si': path_pp+file}

In [5]:
with open('Si.out') as D:
    k = [match for match in D if "!    total energy" in match]
KS_te = str(k).split()[4]

In [1]:
pwd

'/Users/valeria/Documents/aiWT/Final_version/wt/wt/ks_calculations/Si_iso'

In [10]:
def min_energy(x0, *args):
        KS_TE, rho, ions = args
        KE = Functional(type='KEDF',name='WT', rho0=x0)
        grid = DirectGrid(lattice=ions.cell, nr=[len(rho),len(rho),len(rho[0,0])])
        PSEUDO = LocalPseudo(grid = grid, ions=ions, PP_list=PP_list, rcut=10)
        rho_ini = DirectField(grid=grid)
        rho_ini[:] = ions.get_ncharges()/ions.cell.volume
        
        evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)
        optimization_options = {'econv' : 1e-5*ions.nat}
        opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options, 
                           optimization_method = 'TN')
        rho = opt.optimize_rho(guess_rho=rho_ini)
        OF_TE = evaluator.Energy(rho=rho, ions=ions)
        # OF_KE = KE(rho).energy
        diff = np.abs(KS_TE*1/2-OF_TE)
        print('Result', x0, diff)
        return diff

In [11]:
inputfile ='Si.in'
material = read(inputfile, format='espresso-in')
ions = Ions.from_ase(material)

In [12]:
# t = 3
# ions = Ions.from_ase(material)
# cell = ions.get_cell()
# ions.set_cell(cell * t, scale_atoms=True)

In [ ]:
XC = Functional(type='XC',name='LDA')
HARTREE = Functional(type='HARTREE')

rho = io.read_density('rho.xsf')
KS_TE = float(KS_te)
minn = minimize(min_energy, 0.01, args = (KS_TE, rho, ions), method='Powell', bounds=[[0,1]], options={'ftol' : 1e-6})
VOLUME = ions.get_volume()


setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.906673547238E+00     -1.906674E+00   2.917478E-01    1       1       1.021640E-01    
!WARN: Change to steepest decent
1       -3.184709879701E+00     -1.278036E+00   2.140814E+01    1       4       3.846657E-01    
2       -3.734588376627E+00     -5.498785E-01   1.998241E+00    5       1       6.811490E-01    
3       -3.743438962097E+00     -8.850585E-03   8.460049E-01    13      1       1.346237E+00    
4       -3.748398658255E+00     -4.959696E-03   8.872562E-01    12      1       1.955824E+00    
5       -4.025207432906E+00     -2.768088E-01   3.474995E-01    21      5       3.134769E+00    
6       -4.047815491583E+00     -2.260806E-02   1.261596E-01    12      1       3.723026E+00    
7       -4.053836145687E+00     -6.020654E-03   3.222533E-02    15      1       4.465029E+00    
8       -4.0583386

10      -3.427238144465E+00     -6.417659E-04   1.242805E-01    23      1       6.791606E+00    
11      -3.427847939890E+00     -6.097954E-04   1.154796E-01    15      1       7.528259E+00    
12      -3.428433004867E+00     -5.850650E-04   1.151041E-01    15      1       8.311817E+00    
13      -3.428997663155E+00     -5.646583E-04   1.080980E-01    12      1       8.899764E+00    
14      -3.429545041439E+00     -5.473783E-04   1.085495E-01    12      1       9.512571E+00    
15      -3.430077474808E+00     -5.324334E-04   1.027092E-01    19      1       1.043126E+01    
16      -3.430596741553E+00     -5.192667E-04   1.030352E-01    19      1       1.132400E+01    
17      -3.431104195319E+00     -5.074538E-04   9.889110E-02    20      1       1.228800E+01    
18      -3.431601012360E+00     -4.968170E-04   9.901129E-02    20      1       1.322115E+01    
19      -3.432088122988E+00     -4.871106E-04   9.528085E-02    21      1       1.421581E+01    
20      -3.432566283592E+00   

13      -3.544264810179E+00     -1.799983E-04   3.413644E-02    12      1       8.353846E+00    
14      -3.544432580513E+00     -1.677703E-04   3.111923E-02    16      1       9.086614E+00    
15      -3.544590334467E+00     -1.577540E-04   3.040260E-02    17      1       9.876637E+00    
16      -3.544739689303E+00     -1.493548E-04   2.806958E-02    15      1       1.057344E+01    
17      -3.544881883459E+00     -1.421942E-04   2.771361E-02    15      1       1.128874E+01    
18      -3.545017909703E+00     -1.360262E-04   2.582058E-02    18      1       1.212108E+01    
19      -3.545148573619E+00     -1.306639E-04   2.569596E-02    18      1       1.295367E+01    
20      -3.545274554893E+00     -1.259813E-04   2.411138E-02    17      1       1.373008E+01    
21      -3.545396419027E+00     -1.218641E-04   2.414627E-02    12      1       1.429661E+01    
22      -3.545514656492E+00     -1.182375E-04   2.278691E-02    13      1       1.490452E+01    
23      -3.545629678635E+00   

7       -3.670541266356E+00     -7.681750E-03   3.548291E-02    12      1       4.530197E+00    
8       -3.670958589062E+00     -4.173227E-04   2.172583E-02    12      1       5.198115E+00    
9       -3.671079534368E+00     -1.209453E-04   1.361254E-02    14      1       5.952608E+00    
10      -3.677477616401E+00     -6.398082E-03   9.555756E-03    20      5       7.058839E+00    
11      -3.679111632191E+00     -1.634016E-03   4.570129E-03    18      1       7.897757E+00    
12      -3.679426952339E+00     -3.153201E-04   1.753413E-03    17      1       8.778506E+00    
13      -3.679435426358E+00     -8.474019E-06   1.051966E-03    14      1       9.444546E+00    
14      -3.679440899435E+00     -5.473077E-06   9.271789E-04    15      1       1.023486E+01    
15      -3.679445366107E+00     -4.466671E-06   7.498056E-04    14      1       1.097386E+01    
#### Density Optimization Converged ####
Chemical potential (a.u.): -0.14289648998704754
Chemical potential (eV)  : -3.88841155

11      -3.946432092402E+00     -2.217714E-05   4.295462E-03    15      1       8.403950E+00    
12      -3.946452307557E+00     -2.021516E-05   3.514915E-03    12      1       8.967613E+00    
13      -3.946471115512E+00     -1.880795E-05   3.573075E-03    17      1       9.777598E+00    
14      -3.946488818192E+00     -1.770268E-05   3.337721E-03    14      1       1.043040E+01    
15      -3.946505691889E+00     -1.687370E-05   3.267052E-03    23      1       1.154413E+01    
16      -3.946521880146E+00     -1.618826E-05   3.085633E-03    15      1       1.230619E+01    
17      -3.946537485785E+00     -1.560564E-05   3.045739E-03    12      1       1.290953E+01    
18      -3.946552585298E+00     -1.509951E-05   2.895567E-03    14      1       1.356586E+01    
19      -3.946567237931E+00     -1.465263E-05   2.873312E-03    13      1       1.417133E+01    
20      -3.946581490291E+00     -1.425236E-05   2.743467E-03    14      1       1.488167E+01    
21      -3.946595380375E+00   

...

43      -3.816929918074E+00     -3.030080E-05   5.998618E-03    15      1       3.115677E+01    
44      -3.816959962696E+00     -3.004462E-05   5.900596E-03    16      1       3.195801E+01    
45      -3.816989759739E+00     -2.979704E-05   5.901091E-03    13      1       3.269053E+01    
46      -3.817019317001E+00     -2.955726E-05   5.806858E-03    13      1       3.334997E+01    
47      -3.817048641720E+00     -2.932472E-05   5.807639E-03    17      1       3.426213E+01    
48      -3.823294529937E+00     -6.245888E-03   5.146372E-03    18      5       3.525502E+01    
49      -3.823900961406E+00     -6.064315E-04   3.778190E-03    12      1       3.581189E+01    
50      -3.823921055556E+00     -2.009415E-05   2.830220E-03    13      1       3.639957E+01    
51      -3.823934565117E+00     -1.350956E-05   2.201586E-03    14      1       3.709090E+01    
52      -3.823945988752E+00     -1.142364E-05   2.112708E-03    12      1       3.774244E+01    
53      -3.823956061402E+00   

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.906673547238E+00     -1.906674E+00   2.917478E-01    1       1       8.820510E-02    
!WARN: Change to steepest decent
1       -3.092823319356E+00     -1.186150E+00   1.947414E+01    1       4       3.191030E-01    
2       -3.577806762723E+00     -4.849834E-01   1.749139E+00    5       1       5.931461E-01    
3       -3.800484765729E+00     -2.226780E-01   4.399648E-01    20      2       1.553721E+00    
4       -3.811790051608E+00     -1.130529E-02   4.153096E-02    9       1       1.979763E+00    
5       -3.812029099705E+00     -2.390481E-04   2.617126E-02    15      1       2.693389E+00    
6       -3.812153229579E+00     -1.241299E-04   1.728187E-02    12      1       3.262761E+00    
7       -3.819896752198E+00     -7.743523E-03   1.652605E-02    21      5       4.386986E+00    
8       -3.820585588806E+00     -6.888366E-04   5.651964E-03    12      1       4.936581E+00  

8       -3.820441035961E+00     -6.887759E-04   5.647711E-03    12      1       5.769793E+00    
9       -3.821349975285E+00     -9.089393E-04   2.250458E-03    21      2       7.079288E+00    
10      -3.821406829926E+00     -5.685464E-05   1.255601E-03    12      1       7.806107E+00    
11      -3.821622239368E+00     -2.154094E-04   1.026884E-03    14      3       8.716741E+00    
12      -3.821751360490E+00     -1.291211E-04   4.267838E-04    16      1       9.831638E+00    
13      -3.821757788527E+00     -6.428037E-06   2.800609E-04    12      1       1.051431E+01    
14      -3.821759255462E+00     -1.466936E-06   1.570646E-04    14      1       1.128961E+01    
#### Density Optimization Converged ####
Chemical potential (a.u.): -0.14349845420749202
Chemical potential (eV)  : -3.9047918313400514
Result [0.01880566] 9.069537608041855e-06
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
Step    Energy(a.u.)            dE              dP              Nd 

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.906673547238E+00     -1.906674E+00   2.917478E-01    1       1       9.917688E-02    
!WARN: Change to steepest decent
1       -3.092776694194E+00     -1.186103E+00   1.947566E+01    1       4       3.616800E-01    
2       -3.577799140404E+00     -4.850224E-01   1.748920E+00    5       1       6.514170E-01    
3       -3.800350265805E+00     -2.225511E-01   4.398634E-01    20      2       1.739237E+00    
4       -3.811651724917E+00     -1.130146E-02   4.150980E-02    9       1       2.226562E+00    
5       -3.811890664741E+00     -2.389398E-04   2.616037E-02    15      1       3.022880E+00    
6       -3.812014741336E+00     -1.240766E-04   1.727399E-02    12      1       3.650911E+00    
7       -3.819757326469E+00     -7.742585E-03   1.654227E-02    21      5       4.915788E+00    
8       -3.820446040186E+00     -6.887137E-04   5.647247E-03    12      1       5.568550E+00  

8       -3.820450092966E+00     -6.886805E-04   5.646838E-03    12      1       5.559798E+00    
9       -3.821359017419E+00     -9.089245E-04   2.250459E-03    21      2       6.763596E+00    
10      -3.821415908678E+00     -5.689126E-05   1.253862E-03    12      1       7.436777E+00    
11      -3.821631284461E+00     -2.153758E-04   1.023754E-03    14      3       8.286108E+00    
12      -3.821760507703E+00     -1.292232E-04   4.264203E-04    16      1       9.161417E+00    
13      -3.821766939886E+00     -6.432183E-06   2.796918E-04    12      1       9.860779E+00    
14      -3.821768403599E+00     -1.463714E-06   1.570322E-04    14      1       1.062011E+01    
#### Density Optimization Converged ####
Chemical potential (a.u.): -0.14349995124193912
Chemical potential (eV)  : -3.9048325677222833
Result [0.0188051] 7.859939632481883e-08
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
Step    Energy(a.u.)            dE              dP              Nd  

In [23]:
min_energy(0.01, KS_TE, rho, ions)

setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.906673547238E+00     -1.906674E+00   2.917478E-01    1       1       1.240761E-01    
!WARN: Change to steepest decent
1       -3.184709879701E+00     -1.278036E+00   2.140814E+01    1       4       4.205732E-01    
2       -3.734588376627E+00     -5.498785E-01   1.998241E+00    5       1       7.301602E-01    
3       -3.743438962097E+00     -8.850585E-03   8.460049E-01    13      1       1.477929E+00    
4       -3.748398658255E+00     -4.959696E-03   8.872562E-01    12      1       2.190919E+00    
5       -4.025207432906E+00     -2.768088E-01   3.474995E-01    21      5       3.669290E+00    
6       -4.047815491583E+00     -2.260806E-02   1.261596E-01    12      1       4.346727E+00    
7       -4.053836145687E+00     -6.020654E-03   3.222533E-02    15      1       5.154209E+00    
8       -4.0583386

0.23689435856674734

In [24]:
min_energy(0.001, KS_TE, rho, ions)

setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.906673547238E+00     -1.906674E+00   2.917478E-01    1       1       1.077321E-01    
!WARN: Change to steepest decent
1       -3.597610278249E+00     -1.690937E+00   2.942427E+01    1       4       3.980041E-01    
2       -4.359936712398E+00     -7.623264E-01   2.546538E+00    5       1       6.848552E-01    
3       -4.372028079509E+00     -1.209137E-02   1.141446E+00    12      1       1.471137E+00    
4       -4.378278653762E+00     -6.250574E-03   1.049256E+00    13      1       2.146696E+00    
5       -4.383271449169E+00     -4.992795E-03   7.969723E-01    14      1       2.872263E+00    
6       -4.387602629564E+00     -4.331180E-03   8.214628E-01    12      1       3.547161E+00    
7       -4.691347385981E+00     -3.037448E-01   3.753690E-01    18      5       4.700694E+00    
8       -4.7050120

0.8876373479527819

In [25]:
min_energy(0.015, KS_TE, rho, ions)

setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.906673547238E+00     -1.906674E+00   2.917478E-01    1       1       1.443460E-01    
!WARN: Change to steepest decent
1       -3.122509712060E+00     -1.215836E+00   2.013059E+01    1       4       4.645302E-01    
2       -3.630138468246E+00     -5.076288E-01   1.833252E+00    5       1       8.014519E-01    
3       -3.638104636288E+00     -7.966168E-03   7.566654E-01    14      1       1.504148E+00    
4       -3.642601661074E+00     -4.497025E-03   8.125414E-01    14      1       2.200804E+00    
5       -3.882811522910E+00     -2.402099E-01   2.178458E-01    21      6       3.440506E+00    
6       -3.890022578333E+00     -7.211055E-03   2.995567E-02    12      1       4.036959E+00    
7       -3.890187319973E+00     -1.647416E-04   2.059505E-02    13      1       4.679115E+00    
8       -3.8964998

0.07609391881733618

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,3))
im0 = axs[0].plot(np.asarray(vol)*0.529177**3/4,np.asarray(delta_E)*1/4,'*--')
im1 = axs[1].plot(np.asarray(vol)*0.529177**3/4,np.asarray(R0),'*--')
axs[0].set_title('Si-\u03B2$_{tin}$')
axs[1].set_title('Si-\u03B2$_{tin}$')
axs[0].set_xlabel('Volume ($\AA^{3}$)')
axs[0].set_ylabel('$\Delta T_{s}$ (eV/atom)')
axs[1].set_xlabel('Volume ($\AA^{3}$)')
axs[1].set_ylabel('$\u03C1_{0}$')

In [9]:
np.save('/Users/Valeria/Documents/aiWT/Model_ML_function/training_set_data/ks_calculations/Si_8cd/DATA2/rho0_TE', np.asarray(R0))